In [1]:
import numpy as np
from typing import Tuple
from keras.api.models import Sequential
from keras.api.layers import Dense
from keras.api.optimizers import Adam
from collections import deque
from IPython.display import IFrame
import pyautogui
import random
import time
import cv2
from PIL import ImageGrab
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [2]:
def get_player_y():
    screenshot = pyautogui.screenshot()
    img = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)
    
    # Detect red heart (adjust HSV thresholds)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_red = np.array([0, 120, 70])
    upper_red = np.array([10, 255, 255])
    mask = cv2.inRange(hsv, lower_red, upper_red)
    
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        return y + h // 2  # Middle of the heart
    return 0.0  # Fallback  
print()

In [3]:
def get_bone_positions():
    screenshot = pyautogui.screenshot()
    img = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2GRAY)
    _, threshold = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return [cv2.boundingRect(c)[1] for c in contours]  # List of bone y-positions

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size.
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0   # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
    
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def train(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state[np.newaxis, :])[0])
            target_f = self.model.predict(state[np.newaxis, :])
            target_f[0][action] = target
            self.model.fit(state[np.newaxis, :], target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
# You would need to modify the game to:
# 1. Provide state observations (player position, bullet positions, etc.)
# 2. Accept discrete actions (move left, right, up, down, etc.)
# 3. Calculate rewards (survival time, damage dealt, etc.)

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

# Launch Chrome and load the game
driver = webdriver.Chrome()
driver.get("https://jcw87.github.io/c2-sans-fight/")  # Hosted game URL
time.sleep(2)  # Wait for game to load

# Focus the game window
driver.find_element("tag name", "body").click()
webElements = driver.find_element("tag name", "canvas")

In [6]:
def get_google_rect() -> dict[str : Tuple[int, int]]:
        game_rect : dict[str : Tuple[int, int]] = {}

        browser_window = pyautogui.getWindowsWithTitle("Bad Time Simulator")[0]  # or "Chrome" if that's what appears in the title
        
        # Get window coordinates
        x, y = browser_window.left, browser_window.top
        width, height = browser_window.width, browser_window.height
        
        # Adjust these values to capture just the game area (not the browser UI)
        game_x = x + 10  # Add padding to skip browser borders
        game_y = y + 100  # Skip address bar and tabs
        game_width = width - 20
        game_height = height - 120

        game_rect["corrdinates"] = [game_x, game_y]
        game_rect["scale"] = [game_width, game_height]

        return game_rect

In [7]:
def get_game_state():
    player_y = get_player_y()  # From Step 1
    bones = get_bone_positions()  # List of bone y-positions (you'll need to implement this)
    state = np.array([player_y] + bones)
    return np.expand_dims(state, axis=-1)  # Add channel dim for CNN (if needed)

In [8]:
def player_hit(state):
    # Check if player's red heart pixel is missing (got hit)
    return state[player_y, new_player_y] != RED_VALUE

In [9]:
def take_action(action):
    body = driver.find_element("tag name", "body")
    if action == 0:    # Move left
        body.send_keys(Keys.ARROW_LEFT)
    elif action == 1:  # Move right
        body.send_keys(Keys.ARROW_RIGHT)
    elif action == 2:  # Move up
        body.send_keys(Keys.ARROW_UP)
    elif action == 3:  # Move down
        body.send_keys(Keys.ARROW_DOWN)
    elif action == 4:  # Attack (Z key)
        body.send_keys("z")
    elif action == 5:
        pass

In [10]:
def get_reward(state, next_state):
    player_y = state[0]
    new_player_y = next_state[0]
    bones = state[1:]  # Enemy bone positions
    
    # Penalize getting hit
    if check_collision(player_y, bones):
        return -100
    
    # Reward moving closer to safe spots
    closest_bone = min(bones, key=lambda y: abs(y - player_y))
    reward = -abs(player_y - closest_bone) * 0.1
    
    # Small reward for surviving
    return reward + 0.1


In [ ]:
batch_size = 32
episodes = 1000
agent = DQNAgent(state_size = get_bone_positions(), action_size=6)  # Adjust state_size based on bones

for e in range(episodes):
    reset_game()  # You'll need to implement this (e.g., press R key)
    state = get_game_state()
    total_reward = 0
    
    while True:
        action = agent.act(state)
        take_action(action)  # Press UP/DOWN keys
        
        next_state = get_game_state()
        reward = get_reward(state, next_state)
        done = check_game_over()  # Check if player died
        
        agent.remember(state, action, reward, next_state, done)
        total_reward += reward
        state = next_state
        
        if done:
            print(f"Episode: {e}, Reward: {total_reward}")
            break
        
        if len(agent.memory) > batch_size:
            agent.train(batch_size)

ValueError: Invalid dtype: function